In [ ]:
import torch
from torch import nn
from sklearn.datasets import load_boston
#from sklearn.datasets import load_digits

import warnings
import numpy as np

In [ ]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    X,y = load_boston(return_X_y=True)

In [ ]:
#X,y=load_digits(return_X_y=True)

In [ ]:
import pandas as pd  # doctest: +SKIP
import numpy as np

data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
X= np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
y = raw_df.values[1::2, 2]

In [ ]:
X[0]

array([6.320e-03, 1.800e+01, 2.310e+00, 0.000e+00, 5.380e-01, 6.575e+00,
       6.520e+01, 4.090e+00, 1.000e+00, 2.960e+02, 1.530e+01, 3.969e+02,
       4.980e+00])

In [ ]:
print(np.any(X))

True


In [ ]:
X_np = torch.from_numpy(X)

In [ ]:
print(type(X_np))

<class 'torch.Tensor'>


In [ ]:
X_np = X_np.double()

In [ ]:
type(X_np)

torch.Tensor

In [ ]:

import torch.nn.functional as F
class AutoEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(13, 50)
        self.fc2=nn.Linear(50,10)
        self.fc3= nn.Linear(10, 13)
    
    def forward(self, x):
        x = self.fc1(x)
        x=F.relu(x)
        x= self.fc2(x)
        x=F.relu(x)
        x= self.fc3(x)
        return x

In [ ]:
def loss_fn(x_input, x_constructed):                        #x_input and x_constructed here should be tensors here
    loss = torch.mean((x_input-x_constructed)**2)
    return loss

In [ ]:
model = AutoEncoder()
model.double()

AutoEncoder(
  (fc1): Linear(in_features=13, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=13, bias=True)
)

In [ ]:
X0_out = model(X_np[0])

In [ ]:
val = loss_fn(X0_out, X_np[0])

In [ ]:
val

tensor(20325.6335, dtype=torch.float64, grad_fn=<MeanBackward0>)

In [ ]:
optim = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(X, test_size=0.33, random_state=42)

In [ ]:
X_train

array([[1.02330e+01, 0.00000e+00, 1.81000e+01, ..., 2.02000e+01,
        3.79700e+02, 1.80300e+01],
       [6.71910e-01, 0.00000e+00, 8.14000e+00, ..., 2.10000e+01,
        3.76880e+02, 1.48100e+01],
       [1.44550e-01, 1.25000e+01, 7.87000e+00, ..., 1.52000e+01,
        3.96900e+02, 1.91500e+01],
       ...,
       [1.50100e-02, 8.00000e+01, 2.01000e+00, ..., 1.70000e+01,
        3.90940e+02, 5.99000e+00],
       [1.11604e+01, 0.00000e+00, 1.81000e+01, ..., 2.02000e+01,
        1.09850e+02, 2.32700e+01],
       [2.28760e-01, 0.00000e+00, 8.56000e+00, ..., 2.09000e+01,
        7.08000e+01, 1.06300e+01]])

In [ ]:
X_test

array([[9.17800e-02, 0.00000e+00, 4.05000e+00, ..., 1.66000e+01,
        3.95500e+02, 9.04000e+00],
       [5.64400e-02, 4.00000e+01, 6.41000e+00, ..., 1.76000e+01,
        3.96900e+02, 3.53000e+00],
       [1.05740e-01, 0.00000e+00, 2.77400e+01, ..., 2.01000e+01,
        3.90110e+02, 1.80700e+01],
       ...,
       [7.61620e-01, 2.00000e+01, 3.97000e+00, ..., 1.30000e+01,
        3.92400e+02, 1.04500e+01],
       [1.00245e+00, 0.00000e+00, 8.14000e+00, ..., 2.10000e+01,
        3.80230e+02, 1.19800e+01],
       [5.20140e-01, 2.00000e+01, 3.97000e+00, ..., 1.30000e+01,
        3.86860e+02, 5.91000e+00]])

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

batch_size = 64
train_dataloader = DataLoader(X_train, batch_size=batch_size)
test_dataloader = DataLoader(X_test, batch_size=batch_size)

In [ ]:
def train_model(train_dataloder, model, loss_fn, optim, epochs):
    for epoch in range(epochs):
        model.train()
        running_loss = 0
        for i, X in enumerate(train_dataloader):
  

            X1 = model(X)
            loss = loss_fn(X, X1)
            
            optim.zero_grad()
            loss.backward()
            optim.step()

            # print statistics
            running_loss += loss.item()
        print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 10:.3f}')
        running_loss = 0.0
    print("Finished")

In [ ]:
train_model(train_dataloader, model, loss_fn, optim, 10)

[1,     6] loss: 107099.882
[2,     6] loss: 15491.050
[3,     6] loss: 15465.691
[4,     6] loss: 15440.379
[5,     6] loss: 15415.113
[6,     6] loss: 15389.895
[7,     6] loss: 15364.722
[8,     6] loss: 15339.597
[9,     6] loss: 15314.517
[10,     6] loss: 15289.484
Finished


In [ ]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 529 kB 5.2 MB/s 


In [ ]:

def train_network(train_loader, optimizer,criteria, e):
  for epoch in range(e): 

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        
        inputs, labels = data

      
        optimizer.zero_grad()

        outputs = model(inputs)
      
        tmp = torch.nn.functional.one_hot(labels, num_classes= 10)
        loss = criteria(outputs, tmp)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 2000 == 1999:   
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

  print('Finished Training')